### Introduction to PyArrow

*   PyArrow serves as a cross-language development environment specifically designed for in-memory data.
*   Its primary goal is to boost the performance of analytics applications.
*   Emerging from the Apache Arrow project, PyArrow aims to make data interoperability better across different languages and systems.
*   It uses an in-memory columnar data representation, offering an optimized memory footprint for complex data structures.
*   With zero-copy reads, it facilitates quick data sharing between Python and other languages, sidestepping the need for serialization.
*   It supports schemas and metadata, providing data structures that are rich and self-describing.


### PyArrow and Parquet

*   PyArrow offers seamless reading and writing operations for Parquet files.
*   With column pruning, you can selectively read only the necessary columns from a Parquet file, reducing I/O time.


In [ ]:
import pyarrow.parquet as pq 
table = pq.read_table('your_file.parquet', columns=['column1', 'column2']) 
# Potentially conver the file to pandas if needed for more sophisticated splicing and dicing.
df = table.to_pandas()

### Apache Arrow

```The core feature of Apache Arrow is its in-memory columnar format. This language-agnostic standard is designed to store structured, table-like datasets efficiently in memory. The data format supports a rich set of data types, including nested and user-defined types, making it suitable for analytic databases, data frame libraries, and more.``` 

The Apache Arrow Project





<div align="center">
<img src="https://blog.djnavarro.net/posts/2021-11-19_starting-apache-arrow-in-r/img/with_arrow.jpg" width=700>
</div>

[picture source](https://blog.djnavarro.net/posts/2021-11-19_starting-apache-arrow-in-r/)

In [ ]:
# !pip install pyarrow`

### PyArrow Data Structures

*   PyArrow offers a suite of low-level data structures and methods optimized for both speed and flexibility.
*   These structures can be used seamlessly across multiple languages.

### Arrow Array

*   An Arrow Array is essentially a column of data stored in an efficient, contiguous block of memory.
*   Unlike Python lists, these arrays are optimized for high-speed operations and can be transferred across languages without incurring serialization costs.

In [ ]:
import pyarrow as pa
arrow_array = pa.array([1, 2, 3, 4, 5])
print(type(arrow_array))
print("---------")
print(arrow_array)

### Arrow Buffer

* While not a data structure per se, Arrow Buffers are pivotal in understanding Arrow functionality.
* Buffers are blocks of memory that house the data for Arrow Arrays, contributing to efficient storage.
* You can even access the buffer's content directly.




In [ ]:
buffer = arrow_array.buffers()[1]
print(buffer)


In [ ]:
byte_data = buffer.to_pybytes()
print(byte_data)

### Arrow Buffer - Cont'd

* Here, the buffer's data contains 40 bytes, each 8 bytes representing an `int64` value for each of the 5 elements in the array.
* You can use this buffer data to create a new NumPy array, showing that Arrow and NumPy can share memory.




In [ ]:
import numpy as np 
numpy_array = np.frombuffer(buffer, dtype=np.int64)
numpy_array


In [ ]:
np.shares_memory(arrow_array, numpy_array)

### Arrow Buffer - Cont'd

* Both `arrow_array` and `numpy_array` share the same underlying data, demonstrating the concept of zero-copy.
* You can confirm this by modifying a value in one array and seeing the change in the other.
  * Both arrays will now show the updated value.
    
    

In [ ]:
numpy_array[1] = 0
numpy_array

In [ ]:
arrow_array

### Schema

* A schema in PyArrow defines the structure, column names, and types for Arrow Arrays.
* Schemas are crucial as they set the framework for data manipulation and operations in Arrow.
  * Give Arrow an idea on how to encode the data



In [ ]:
schema = pa.schema([('column1', pa.int64()), ('column2', pa.string())])
print(schema)

### Chunked Array

*   A Chunked Array in PyArrow is like a single Arrow Array but divided into smaller "chunks."
*   This structure allows for the storage and processing of datasets that are too large to fit in memory.
*   It's commonly used in distributed computing frameworks and data streaming scenarios.

* For example:
  * you could have data sent in chunks to optimize throughput
  * you might have multiple nodes in a distributed system each producing Arrow Arrays that are collected and represented as a ChunkedArray by the master node.

* From a user perspective, a Chunked Array appears as a contiguous sequence of data.




In [ ]:
results_node_1 = pa.array([0,1,2,3,4])
results_node_2 = pa.array([5,6,7,8,9,10])
chunked_array = pa.chunked_array([results_node_1, results_node_2])
chunked_array


### Chunked Array - Cont'd

* You can index into a single position or even across multiple chunks, making the data handling more versatile.
* You can also access individual chunks, allowing for parallel processing.

In [ ]:
chunked_array[3:6]

In [ ]:
chunked_array.chunk(0)

### Table

* A Table in PyArrow is a container for multiple Arrow ChunkedArrays with a common schema.
* Each column in the Table is an Arrow ChunkedArray, and all columns share the same length.
* Tables offer an ideal format for handling data in the form of a dataframe.
* Tables can also be partitioned across multiple files for large-scale storage, or to be sent across a network, or even to be stored in-memory on a single machine.






In [ ]:
column1 = pa.array([0, 1, 2, 3, 4]) 
column2 = pa.array(['a', 'b', 'c', 'd', 'e'])
table = pa.table({'column1': column1, 'column2': column2})  

table

### Record Batch

*   A Record Batch is a collection of Arrow Arrays (columns) with the same length, all of which are bundled together with a schema.
*   Much like a Chunked Array is a collection of Arrow Arrays, a Table in Apache Arrow is a collection of Record Batches.

* Conceptual Relationship
  *   In Apache Arrow, the concept of a Record Batch is to a Table what an Arrow Array is to a Chunked Array.
    *   Arrays can be grouped together to form a Chunked Array.
    *   Record Batches can be grouped together to form a Table.




### Record Batch - Cont'd

* Use Cases
  *   The choice between using a Record Batch or a Table often depends on your specific needs. E.g.:
    
  *  Streaming Data: If you need to process data on-the-fly, perhaps in a streaming application where you want to process each chunk as it arrives, Record Batches are a good choice.
    *   You can serialize and process each Record Batch independently as they arrive, without having to wait for the entire data set.


In [ ]:

column1_array = pa.array([1, 2, 3, 4, 5])
column2_array = pa.array(['a', 'b', 'c', 'd', 'e'])
schema = pa.schema([('column1', pa.int64()), ('column2', pa.string())])

record_batch = pa.record_batch([column1_array, column2_array], schema=schema)
record_batch


In [ ]:
record_batch.columns

In [ ]:
record_batch["column1"]

In [ ]:

column1_array_new = pa.array([6, 7, 8, 9, 10])
column2_array_new = pa.array(['f', 'g', 'h', 'i', 'j'])
record_batch_new = pa.record_batch([column1_array_new, column2_array_new], schema=schema)


table = pa.Table.from_batches([record_batch, record_batch_new], schema=schema)
table


### Record Batch - Cont'd

* In the example above, two Record Batches are combined to create a single Table. 
  * This is analogous to how individual Arrow Arrays can be combined to create a Chunked Array
  * Reinforces the idea that a Record Batch is to a Table what an Arrow Array is to a Chunked Array.


### Dive Into Real Data: Parquet and Memory Efficiency

1.  Let's get hands-on and read a Parquet file using Apache Arrow.
2.  Take note: the size of the data when using PyArrow is substantially smaller than a Pandas DataFrame for the same data.
3.  Think of this as a little teaser to whet your appetite for data science goodness.

**Note**: Here, I'm using the `parquet` module from the PyArrow package. This module knows how to read Parquet files among other things.



In [ ]:
import pyarrow.parquet as pq
table = pq.read_table('/Users/mahdi/Downloads/fhvhv_tripdata_2022-06.parquet')
table


In [ ]:
import sys
sys.getsizeof(table) / 1024 / 1024 / 1024

In [ ]:
# import os
# import psutil
# def print_mem():
#     gig = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 3
#     print(f"{gig} gigabytes")

# print_mem()


In [ ]:
import pandas as pd
df = pd.read_parquet('/Users/mahdi/Downloads/fhvhv_tripdata_2022-06.parquet')
sys.getsizeof(df) / 1024 / 1024 / 1024



### Apache Arrow Datasets


*   Datasets in PyArrow let you work with large tabular data, even when it's larger than your machine's memory
*   It offers lazy data access, meaning you don't have to load the entire dataset into memory.
*   Datasets support data discovery, partitioning, and compatibility with various file systems like AWS, Google Cloud, and local storage.
  * I can read from AWS or Google without having to install anything.

* import the dataset library as:

```python
import pyarrow.dataset as ds
```


### Dataset Overview

* Provider: New York City Taxi and Limousine Commission (TLC)
* Data hosted on AWS. The URSA-LAB company account.
* Contains data on millions of taxi and limousine trips in NYC
* Time Period: 2009 to 2019


In [ ]:
# **Note**: In the AWS S3 listing, "PRE" stands for "prefix," essentially representing a folder or directory.

!aws s3 ls "s3://ursa-labs-taxi-data/"

In [ ]:
!aws s3 ls "s3://ursa-labs-taxi-data/2009/"

In [ ]:
%%time
import pyarrow.dataset as ds
dataset = ds.dataset("s3://ursa-labs-taxi-data/", partitioning=["year", "month"])
dataset

In [ ]:
len(dataset.files)

In [ ]:
dataset.files[0:10]

In [ ]:
# Here's how to load just one file (a fragment) and its schema:

frag = next(dataset.get_fragments())
frag.partition_expression

#### Play with a Single File

* Let's read in the data from this single fragment
* Take a look at the data
* List of column names
    

In [ ]:
%%time
frag_table = frag.to_table()
frag_table

In [ ]:
frag_table.column_names

In [ ]:
frag_table.num_rows


In [ ]:
frag_table

#### Chunks: The Building Blocks

* Remember how we talked about Arrow tables having columns that could be split into chunks? 
* If you take a look, each column is divided into 216 chunks
  * Proving that this table is built in the way we discussed earlier.
* Take just a slice of the data.

In [ ]:
frag_table.slice(0, 5)

In [ ]:
[frag_table[col_name].num_chunks for col_name in frag_table.column_names]


### The Essentials of Apache Arrow Tables and Record Batches

*  Tables in Apache Arrow are essentially collections of record batches.
*  You can easily pull data from columns like `payment_type`, `fare_amount`, or `tip_amount`. 
* Because we're working with a single record batch, managing the data is pretty straightforward. 
  * We'll see that each column, for instance, holds 65,536 values.


In [ ]:
record_batch_3 = frag_table.to_batches()[3]
record_batch_3

In [ ]:
record_batch_3.num_rows

In [ ]:
record_batch_3["fare_amount"]

In [ ]:
record_batch_3['tip_amount']

In [ ]:
record_batch_3['payment_type']

#### PyArrow's Computational Capabilities

*   PyArrow separates data storage concerns from computational functionality.    
    * Structures like Arrow Arrays, Record Batches, and Tables handle data storage and serialization.
    * For actual data operations, there's the `pyarrow.compute` module.
*   The `pyarrow.compute` module offers a range of functions for filtering, transforming, and aggregating data.    
    * While it does provide useful operations, it's not a full-blown analytical tool. 
    * For more complex tasks, you'd typically use something like Pandas or Spark.

* Let's perform some computations like calculating the sum of tips and fares, etc.


In [ ]:
import pyarrow.compute as pc
pc.add(record_batch_3['tip_amount'], record_batch_3['fare_amount'])

* How about finding the maximum total amount for a trip, including the tip?

In [ ]:
pc.max(pc.add(record_batch_3['tip_amount'], record_batch_3['fare_amount']))

* And the average?


In [ ]:
pc.mean(pc.add(record_batch_3['tip_amount'], record_batch_3['fare_amount']))

* We can also perform operations on string data, like converting the case of `payment_type`, which has been recorded inconsistently.


In [ ]:
upper_cased_payment_type = pc.utf8_upper(record_batch_3["payment_type"])
upper_cased_payment_type

* You can then filter data based on whether the payment type was "CASH."


In [ ]:
is_cash = pc.equal(upper_cased_payment_type, pa.scalar('CASH'))
is_cash 

In [ ]:
filtered_record_batch_3 = pc.filter(record_batch_3, is_cash)
filtered_record_batch_3
filtered_record_batch_3.num_rows


#### Working with Parquet Files

* You can read Parquet data into PyArrow as a ParquetDataset, and then work with it as ParquetFile Fragments.
* Recall that: 
    * Each fragment has its own metadata, 
    * You can also get statistics about each row group within the fragment.
      * However, it's usually more efficient to work with sorted data if you carry out frequent operations
      * You can then save this sorted table into a new Parquet file for optimized data retrieval.

In [ ]:
import pyarrow as pa 
import pyarrow.parquet as pq
dataset = pq.ParquetDataset('s3://ursa-labs-taxi-data/2009/', partitioning=["month"])
dataset

In [ ]:
%%time 
data_table = dataset.fragments[0].to_table() 
sorted_indices = pc.sort_indices(data_table, sort_keys=[("dropoff_at", "ascending"), ("fare_amount", "ascending")])
sorted_indices

In [ ]:
# takes the instances in the order specified in the variable sorted_indices
# i.e., sorting the data
sorted_table = data_table.take(sorted_indices)

In [ ]:
# pq.write_table(sorted_table, 'optimized_parquet_file.parquet', row_group_size=65536)


#### Exploring Sorted Parquet Files

*   When you read the sorted table back into PyArrow, it's easier to work with.
  * We can reach the read groups meta data and only look at those we are interested in.
  * i.e., you can delve into the metadata to understand your data better.



In [ ]:
optimized_parquet_file = pq.ParquetFile('optimized_parquet_file.parquet')
rg0_metadata = optimized_parquet_file.metadata.row_group(0)
rg0_metadata_dict = rg0_metadata.to_dict()

In [ ]:
[(i,x["path_in_schema"]) for i, x in enumerate(rg0_metadata.to_dict()["columns"])]

In [ ]:
name_2_pos = {x["path_in_schema"]:i for i, x in enumerate(rg0_metadata.to_dict()["columns"])}
name_2_pos

# Bonus Question Answers

In [6]:
import pyarrow as pa 
import pyarrow.parquet as pq
import pyarrow.compute as pc

In [3]:
optimized_parquet_file = pq.ParquetFile('optimized_parquet_file.parquet')
rg0_metadata = optimized_parquet_file.metadata.row_group(0)
name_2_pos = {x["path_in_schema"]:i for i, x in enumerate(rg0_metadata.to_dict()["columns"])}

In [4]:
from datetime import datetime
col_idx = name_2_pos['dropoff_at']

datetime_obj_start = datetime.strptime("2009-1-1 14:00:00", "%Y-%m-%d %H:%M:%S")
datetime_obj_end = datetime.strptime("2009-1-1 14:59:00", "%Y-%m-%d %H:%M:%S")

for i in range(optimized_parquet_file.num_row_groups):
    col_stats = optimized_parquet_file.metadata.row_group(i).column(col_idx).statistics    
    if col_stats.min <= datetime_obj_start <= col_stats.max:
        print(f"Start point (2:00 PM) found in row_group {i}")
        
    if col_stats.min <= datetime_obj_end <= col_stats.max:
        print(f"Ending point (2:59 PM) found in row_group {i}")

    

Start point (2:00 PM) found in row_group 2
Ending point (2:59 PM) found in row_group 2


### Bonus Question 1
    *  can you get the average transaction between 2:00-2:59 PM

In [7]:
# Considering "Dropoff_at" instead of "pickup_at" as the transaction takes place after the dropoff

rg2 = optimized_parquet_file.read_row_group(2)

#Drop-off time >= 2:00 PM
gr_2_00PM = pc.greater_equal(rg2['dropoff_at'], datetime_obj_start)

#Drop-off time <= 2:59 PM
ls_2_59PM = pc.less_equal(rg2['dropoff_at'], datetime_obj_end)

In [8]:
#filter the data based on the above two conditions
all_drop_off_in_range = pc.filter(rg2, pc.and_(gr_2_00PM, ls_2_59PM))

In [9]:
all_drop_off_in_range['total_amount']

[
  [
    2.5,
    2.9,
    3.3,
    3.3,
    3.3,
    ...
    49.15,
    49.15,
    49.15,
    45,
    47.7
  ]
]

In [10]:
pc.mean(all_drop_off_in_range['total_amount'])

<pyarrow.DoubleScalar: 10.370755473289336>

    average transaction between 2:00-2:59 PM on 2009-1-1 was $10.37

### Bonus Question 2
        * Which day, on average has the highest tip? 

In [11]:
from tqdm import tqdm
import pandas as pd
from collections import defaultdict

In [12]:
tip_per_day = defaultdict(float)
count_per_day = defaultdict(int)

# loop over all row groups
for i in tqdm(range(optimized_parquet_file.num_row_groups)):

    #convert each row group to pandas dataframe
    rg = optimized_parquet_file.read_row_group(i).to_pandas()
    # convert the pickup_at column to string and extract the date in yyyy-mm-dd format 
    rg['pickup_at'] = rg['pickup_at'].astype('str')
    rg['pickup_at'] = rg['pickup_at'].str.split(' ', expand=True)[0]
    
    # sum all the tips for each day and count the number of trips for each day
    total_tip = rg.groupby('pickup_at')['tip_amount'].sum()
    counts = rg.groupby('pickup_at')['pickup_at'].count()

    # add the tip and count for each day to the defaultdict
    for date, tip in total_tip.items():
        tip_per_day[date] += tip
        count_per_day[date] += counts[date]

# round the total to 2 decimal places
tip_per_day = {k: round(v, 2) for k, v in tip_per_day.items()}

  0%|          | 0/216 [00:00<?, ?it/s]

100%|██████████| 216/216 [00:20<00:00, 10.46it/s]


In [13]:
# Calculate the avaerage tip for each day
avg = {}
for k, v in tip_per_day.items():
    avg[k] = round(v / count_per_day[k], 2)

In [14]:
# sort the avg by values in descending order to get the max avg tip
# Can be done efficiently without sorting, but this was done just for curiosity
sorted_avg = sorted(avg.items(), key=lambda x: x[1], reverse=True)

In [15]:
print('\t\t\tsorted by avg tip per day\n')

print(f'Date\t\t|\tTotal\t\t|\tCount\t|\tAverage')
print('------------------------------------------------------------------------')
for row in sorted_avg:
    print(f'{row[0]}\t|\t{tip_per_day[row[0]]}\t|\t{count_per_day[row[0]]}\t|\t{row[1]}')

			sorted by avg tip per day

Date		|	Total		|	Count	|	Average
------------------------------------------------------------------------
2009-01-29	|	267373.54	|	503352	|	0.53
2009-01-15	|	251352.51	|	486450	|	0.52
2009-01-21	|	242664.74	|	478268	|	0.51
2009-01-22	|	255874.08	|	498241	|	0.51
2009-01-28	|	233095.36	|	455187	|	0.51
2009-01-14	|	246874.6	|	489177	|	0.5
2009-01-20	|	216461.05	|	433639	|	0.5
2009-01-27	|	235145.65	|	471788	|	0.5
2009-01-30	|	267957.11	|	538365	|	0.5
2009-01-13	|	216002.36	|	442543	|	0.49
2009-01-19	|	173412.32	|	352534	|	0.49
2009-01-23	|	254119.05	|	519394	|	0.49
2009-01-16	|	259469.1	|	535200	|	0.48
2009-01-25	|	220655.79	|	460723	|	0.48
2009-01-26	|	210454.43	|	434081	|	0.48
2009-01-11	|	191854.23	|	405075	|	0.47
2009-01-12	|	195425.52	|	414642	|	0.47
2009-01-08	|	220165.2	|	477502	|	0.46
2009-01-04	|	166291.08	|	367525	|	0.45
2009-01-07	|	168789.04	|	371043	|	0.45
2009-01-09	|	232536.04	|	520846	|	0.45
2009-01-18	|	189843.75	|	419962	|	0.45
2009-01-05	|	

    On Average 2009-01-29 had the most ammount of tip ($0.53/trip) and 2009-01-01 had the lowest ($0.32/trip) in the month of January in 2009.

### Bonus Question 3
        * Which time (hour) of the day has the highest tip?
        Might be interpreted in 3 different ways.
                1. Generally Which hour of the days brought in more tip in January, 2009
                2. Consider it linked with Question 2: 
                    Which hour of the 2009-01-29 brought in most of the tip
                3. Consider it linked with Question 1: 
                    Which hour of the 2009-01-1 brought in most of the tip

    1. Generally Which hour of the days brought in more tips in January, 2009

In [16]:
tips_bucket = {}
count_bucket = {}

for i in tqdm(range(optimized_parquet_file.num_row_groups)):
    rg = optimized_parquet_file.read_row_group(i).to_pandas()
    # convert the pickup_at column to string and extract the date in yyyy-mm-dd format 
    rg['pickup_at'] = rg['pickup_at'].astype('str')
    rg['pickup_at_hour'] = rg['pickup_at'].str.split(' ', expand=True)[1]
    rg['pickup_at_hour'] = rg['pickup_at_hour'].str.split(':', expand=True)[0].astype('int')
    rg['pickup_at'] = rg['pickup_at'].str.split(' ', expand=True)[0]
    
    # group by the hour and find the max tip for each hour
    group = rg.groupby('pickup_at_hour')['tip_amount'].max().sort_values(ascending=False)
    count = rg.groupby('pickup_at_hour')['pickup_at_hour'].count()
    for key in group.index:
        if key not in tips_bucket.keys():
            tips_bucket[key] = group[key]
            count_bucket[key] = count[key]
        else:
            tips_bucket[key] += group[key]
            count_bucket[key] += count[key]


tips_bucket = {k: round(v, 2) for k, v in tips_bucket.items()}

100%|██████████| 216/216 [00:45<00:00,  4.75it/s]


In [19]:
# Calculate the avaerage tip for each day
avg_tips_hour = {}
for k, v in tips_bucket.items():
    avg_tips_hour[k] = round((v / count_bucket[k])*100, 2)

In [25]:
sorted_avg_hour = sorted(avg_tips_hour.items(), key=lambda x: x[1], reverse=True)
sorted_tips_bucket = sorted(tips_bucket.items(), key=lambda x: x[1], reverse=True)

In [26]:
sorted_tips_bucket

[(15, 2986.03),
 (14, 2790.38),
 (12, 2765.73),
 (16, 2743.74),
 (22, 2668.85),
 (20, 2668.47),
 (0, 2597.99),
 (18, 2573.61),
 (19, 2540.96),
 (17, 2519.28),
 (21, 2511.47),
 (23, 2417.89),
 (13, 2394.76),
 (9, 2393.78),
 (7, 2250.11),
 (8, 2219.56),
 (1, 2217.7),
 (10, 2163.76),
 (11, 2146.86),
 (3, 1896.05),
 (4, 1871.07),
 (2, 1685.02),
 (6, 1650.61),
 (5, 1596.38)]

In [24]:
print('\t\t\tsorted by avg tip per hour\n')

print(f'Hour\t|\tTotal\t|\tCount\t|\tAverage')
print('------------------------------------------------------------------------')
for row in sorted_avg_hour:
    print(f'{row[0]}\t|\t{tips_bucket[row[0]]:.2f}\t|\t{count_bucket[row[0]]}\t|\t{row[1]}')

			sorted by avg tip per hour

Hour	|	Total	|	Count	|	Average
------------------------------------------------------------------------
5	|	1596.38	|	123974	|	1.29
4	|	1871.07	|	160377	|	1.17
3	|	1896.05	|	225803	|	0.84
6	|	1650.61	|	265589	|	0.62
1	|	2217.70	|	394213	|	0.56
2	|	1685.02	|	303536	|	0.56
0	|	2597.99	|	529466	|	0.49
7	|	2250.11	|	485629	|	0.46
15	|	2986.03	|	708964	|	0.42
12	|	2765.73	|	671394	|	0.41
16	|	2743.74	|	668310	|	0.41
14	|	2790.38	|	702784	|	0.4
9	|	2393.78	|	642209	|	0.37
23	|	2417.89	|	676273	|	0.36
10	|	2163.76	|	595728	|	0.36
13	|	2394.76	|	674441	|	0.36
11	|	2146.86	|	613942	|	0.35
8	|	2219.56	|	640186	|	0.35
22	|	2668.85	|	767651	|	0.35
17	|	2519.28	|	766431	|	0.33
21	|	2511.47	|	789349	|	0.32
20	|	2668.47	|	830731	|	0.32
18	|	2573.61	|	913010	|	0.28
19	|	2540.96	|	942423	|	0.27


    Through January, 2009 most tips (on average) were generated during the night - from 12 AM to 7 AM with 5AM having the most. 
    However if we consider the total amount of tips, the noon-time (12 PM to 4 PM) seems more generous with 3 PM having the most amount of tips.
    -------------------------------------------------------------

    2. Which hour of the 2009-01-29 brought in most of the tips

In [60]:
tips_bucket = {}
count_bucket = {}

for i in tqdm(range(optimized_parquet_file.num_row_groups)):
    rg = optimized_parquet_file.read_row_group(i).to_pandas()
    # convert the pickup_at column to string and extract the date in yyyy-mm-dd format 
    rg['pickup_at'] = rg['pickup_at'].astype('str')
    rg['pickup_at_hour'] = rg['pickup_at'].str.split(' ', expand=True)[1]
    rg['pickup_at_hour'] = rg['pickup_at_hour'].str.split(':', expand=True)[0].astype('int')
    rg['pickup_at'] = rg['pickup_at'].str.split(' ', expand=True)[0]
    rg = rg[rg['pickup_at'] == '2009-01-29']
    
    # group by the hour and find the max tip for each hour
    group = rg.groupby('pickup_at_hour')['tip_amount'].max().sort_values(ascending=False)
    count = rg.groupby('pickup_at_hour')['pickup_at_hour'].count()
    for key in group.index:
        if key not in tips_bucket.keys():
            tips_bucket[key] = group[key]
            count_bucket[key] = count[key]
        else:
            tips_bucket[key] += group[key]
            count_bucket[key] += count[key]


tips_bucket = {k: round(v, 2) for k, v in tips_bucket.items()}

100%|██████████| 216/216 [00:52<00:00,  4.11it/s]


In [61]:
# Calculate the avaerage tip for each day
avg_tips_hour = {}
for k, v in tips_bucket.items():
    avg_tips_hour[k] = round((v / count_bucket[k])*100, 2)

In [62]:
sorted_avg_hour = sorted(avg_tips_hour.items(), key=lambda x: x[1], reverse=True)
sorted_tips_bucket = sorted(tips_bucket.items(), key=lambda x: x[1], reverse=True)

In [64]:
sorted_tips_bucket

[(0, 123.2),
 (8, 118.54),
 (22, 114.04),
 (19, 105.27),
 (15, 101.65),
 (12, 100.99),
 (7, 91.94),
 (20, 90.44),
 (9, 85.05),
 (14, 84.12),
 (17, 83.33),
 (18, 79.57),
 (10, 78.0),
 (23, 76.96),
 (13, 76.9),
 (16, 74.34),
 (21, 72.32),
 (11, 66.31),
 (1, 58.23),
 (6, 48.0),
 (3, 37.5),
 (5, 29.86),
 (2, 26.0),
 (4, 16.6)]

In [63]:
print('\t\tsorted by avg tip per hour in 2009-01-29\n')

print(f'Hour\t|\tTotal\t|\tCount\t|\tAverage')
print('------------------------------------------------------------------------')
for row in sorted_avg_hour:
    print(f'{row[0]}\t|\t{tips_bucket[row[0]]:.2f}\t|\t{count_bucket[row[0]]}\t|\t{row[1]}')

		sorted by avg tip per hour in 2009-01-29

Hour	|	Total	|	Count	|	Average
------------------------------------------------------------------------
3	|	37.50	|	3202	|	1.17
0	|	123.20	|	12450	|	0.99
1	|	58.23	|	7322	|	0.8
5	|	29.86	|	3813	|	0.78
4	|	16.60	|	2549	|	0.65
2	|	26.00	|	4648	|	0.56
12	|	100.99	|	23529	|	0.43
6	|	48.00	|	11504	|	0.42
7	|	91.94	|	22543	|	0.41
8	|	118.54	|	29231	|	0.41
15	|	101.65	|	24679	|	0.41
22	|	114.04	|	30689	|	0.37
14	|	84.12	|	24318	|	0.35
10	|	78.00	|	22844	|	0.34
13	|	76.90	|	22505	|	0.34
16	|	74.34	|	21584	|	0.34
17	|	83.33	|	25994	|	0.32
9	|	85.05	|	27205	|	0.31
11	|	66.31	|	21904	|	0.3
19	|	105.27	|	34945	|	0.3
23	|	76.96	|	27584	|	0.28
20	|	90.44	|	33386	|	0.27
18	|	79.57	|	32552	|	0.24
21	|	72.32	|	32372	|	0.22


    On 2009-01-29, on average most tips were geenrated at 3 AM. The night time (12 AM - 8 AM) was the best for Tips on average.
    When considering the total, During the evening, midnight and the noon-time there were spikes in Tips, 12 AM being the best.
    ------------------------------------------------------------------

    3. Which hour of the 2009-01-01 brought in most of the tips

In [65]:
tips_bucket = {}
count_bucket = {}

for i in tqdm(range(optimized_parquet_file.num_row_groups)):
    rg = optimized_parquet_file.read_row_group(i).to_pandas()
    # convert the pickup_at column to string and extract the date in yyyy-mm-dd format 
    rg['pickup_at'] = rg['pickup_at'].astype('str')
    rg['pickup_at_hour'] = rg['pickup_at'].str.split(' ', expand=True)[1]
    rg['pickup_at_hour'] = rg['pickup_at_hour'].str.split(':', expand=True)[0].astype('int')
    rg['pickup_at'] = rg['pickup_at'].str.split(' ', expand=True)[0]
    rg = rg[rg['pickup_at'] == '2009-01-01']
    
    # group by the hour and find the max tip for each hour
    group = rg.groupby('pickup_at_hour')['tip_amount'].max().sort_values(ascending=False)
    count = rg.groupby('pickup_at_hour')['pickup_at_hour'].count()
    for key in group.index:
        if key not in tips_bucket.keys():
            tips_bucket[key] = group[key]
            count_bucket[key] = count[key]
        else:
            tips_bucket[key] += group[key]
            count_bucket[key] += count[key]


tips_bucket = {k: round(v, 2) for k, v in tips_bucket.items()}

100%|██████████| 216/216 [00:52<00:00,  4.13it/s]


In [66]:
# Calculate the avaerage tip for each day
avg_tips_hour = {}
for k, v in tips_bucket.items():
    avg_tips_hour[k] = round((v / count_bucket[k])*100, 2)

In [67]:
sorted_avg_hour = sorted(avg_tips_hour.items(), key=lambda x: x[1], reverse=True)
sorted_tips_bucket = sorted(tips_bucket.items(), key=lambda x: x[1], reverse=True)

In [68]:
sorted_tips_bucket

[(20, 121.56),
 (16, 115.0),
 (12, 113.23),
 (1, 101.23),
 (3, 67.56),
 (11, 66.7),
 (4, 65.49),
 (14, 62.0),
 (6, 58.88),
 (21, 56.66),
 (18, 53.95),
 (13, 53.5),
 (19, 53.25),
 (23, 49.29),
 (10, 42.66),
 (15, 41.0),
 (9, 40.0),
 (22, 39.25),
 (17, 34.18),
 (0, 33.8),
 (2, 30.0),
 (7, 25.0),
 (8, 25.0),
 (5, 23.52)]

In [69]:
print('\t\tsorted by avg tip per hour in 2009-01-01\n')

print(f'Hour\t|\tTotal\t|\tCount\t|\tAverage')
print('------------------------------------------------------------------------')
for row in sorted_avg_hour:
    print(f'{row[0]}\t|\t{tips_bucket[row[0]]:.2f}\t|\t{count_bucket[row[0]]}\t|\t{row[1]}')

		sorted by avg tip per hour in 2009-01-01

Hour	|	Total	|	Count	|	Average
------------------------------------------------------------------------
6	|	58.88	|	3682	|	1.6
9	|	40.00	|	3289	|	1.22
8	|	25.00	|	2602	|	0.96
7	|	25.00	|	2899	|	0.86
20	|	121.56	|	17102	|	0.71
12	|	113.23	|	16523	|	0.69
4	|	65.49	|	9619	|	0.68
1	|	101.23	|	16990	|	0.6
16	|	115.00	|	19221	|	0.6
3	|	67.56	|	13307	|	0.51
11	|	66.70	|	13366	|	0.5
10	|	42.66	|	9610	|	0.44
5	|	23.52	|	5391	|	0.44
23	|	49.29	|	13494	|	0.37
21	|	56.66	|	16847	|	0.34
14	|	62.00	|	19283	|	0.32
13	|	53.50	|	18082	|	0.3
18	|	53.95	|	19860	|	0.27
19	|	53.25	|	20140	|	0.26
22	|	39.25	|	15975	|	0.25
0	|	33.80	|	15386	|	0.22
15	|	41.00	|	19809	|	0.21
2	|	30.00	|	15585	|	0.19
17	|	34.18	|	19563	|	0.17


    On 2009-01-01, on average most tips were geenrated at 6 AM. The morning (6 AM - 9 AM) was the best for Tips on average.
    When considering the total, During the evening through midnight the Tips were the most with 8PM at the top. During the Noon (at 12 PM) there was a spike in tips as well.


### Resources

1.  [Apache Arrow Homepage](https://arrow.apache.org/)
2.  [PyArrow Documentation](https://arrow.apache.org/docs/python/)
3.  [PyArrow GitHub Repository](https://github.com/apache/arrow/tree/master/python/pyarrow)